# Neural Machine Translation - Assignment 2

In this task you will develop a neural machine translation (NMT) system to translate text from one language to another. For this, you wil need to chose the data to train the models, perform data processing and train a sequence2sequence neural model.


## Section 1- Data Collection and Preprocessing 


---


**Task 1  (5 marks)**

---

There are few datasets to train an NMT system available from Tatoeba Project (http://www.manythings.org/anki/) or OPUS project (http://opus.nlpl.eu/).

*  Download a langauge pair (preferably European language) and **extract** the file(s) and upload it to colab
*  Create a list of lines by splitting the text file at every occurance of '\n'
*  Print number of sentences
*  Limit the amount of senteces to 10,000 lines (but more than 5,000 lines)
*  Split the data into train and test [You can split validation set here or while training use kerase validation_split option]
*  Print 100th sentence in original script[ not unicode] for source and target language


In [62]:
#your code goes here
import numpy as np
import keras, tensorflow
import gensim
from keras.models import Model

from keras.layers import Input, LSTM, Dense
from google.colab import drive
drive.mount('/content/drive')
filename="/content/drive/My Drive/Colab Notebooks/fra.txt"
with open(filename , 'r') as dataset:
  lines=dataset.read().split('\n')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
print(len(lines))
print(lines[100])

167131
Come in.	Entrez !


**Task 2 (5 marks)** 

---

* Add '\t' to denote begining of sentence and '\n'  or '<eos\>' to denote end of the sentence to the each target line.
* Preprocess (word tokenisation, lowecasing) the text.

In [0]:
# your code goes here
import nltk
#nltk.download('all')
# set the sample number is 8000
sampes_num=8000
input_texts=[]
target_texts=[]
source_tokenize=[]
target_tokenize=[]
Source_vocabulary=set()
target_vocabulary=set()
#if the length of samples < 8000, get all text sentence, otherwise 
#get 8000 sentence
samples=lines[0:min(8000,len(lines)-1)]
for line in samples:
    source_data, target_data=line.split('\t')
#     target_data='\t'+target_data+'\n'
    #tokenize get all words in the sentence
    source_tokenize=nltk.word_tokenize(source_data)
    target_tokenize=nltk.word_tokenize(target_data)
    #add \t and \n on the head and end
    target_tokenize.insert(0,'\t')
    target_tokenize.insert(len(target_tokenize),'\n')
    #get the vocabulary for source and target
    for i in source_tokenize:
       Source_vocabulary.add(i.lower())
    for i in target_tokenize:
      target_vocabulary.add(i.lower())       
    #append token to input text
    input_texts.append(source_tokenize)
    #append token to target text
    target_texts.append(target_tokenize)
#the max encoder and decoder sequence length is get the max length in the total corpus 
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])  
#get the encoder and decoder vocabulary length
num_encoder_tokens=len(Source_vocabulary)
num_decoder_tokens=len(target_vocabulary)

In [65]:
print('Number of samples:', len(input_texts))
print('Number of unique source language tokens:', num_encoder_tokens)
print('Number of unique target language tokens:', num_decoder_tokens)
print('Max sequence length of source language:', max_encoder_seq_length)
print('Max sequence length of target language:', max_decoder_seq_length)
print("Source Vocabulary",Source_vocabulary)
print("Target Vocabulary",target_vocabulary)

Number of samples: 8000
Number of unique source language tokens: 1833
Number of unique target language tokens: 3948
Max sequence length of source language: 6
Max sequence length of target language: 13
Source Vocabulary {'short', 'warmer', 'off-duty', 'manage', 'trains', 'powerless', 'mishap', 'baked', 'six', 'exercised', 'bought', 'afternoon', 'were', 'door', 'helped', 'serious', 'creepy', 'lock', 'file', 'cranky', 'where', 'bounce', 'managing', 'dislike', 'mondays', 'maid', 'despair', 'hurry', 'lift', 'deaf', 'fear', 'vote', 'judge', 'finished', 'candy', 'disagreed', 'meet', 'cake', 'worked', 'braces', 'ached', 'seriously', 'kissed', 'goes', 'table', 'whining', 'ramble', 'adults', 'ok', "n't", 'was', 'drank', 'ashamed', 'outraged', 'offer', 'boats', 'fish', 'die', 'cash', 'go', 'weighed', 'remember', 'scary', 'gear', 'cop', 'pooped', 'rush', 'birds', 'hint', 'painted', 'date', 'guess', 'feed', 'recycle', 'laugh', 'trips', 'slow', 'dj', 'men', 'say', 'outgoing', 'medic', 'smart', "'ve"

** Task 3 (10 marks)**

---

*  Assign each unique word to an integer value (5 marks).
*  Create word embedding for your vocabulary using pretrained Glove embeddigns (5 marks) (http://nlp.stanford.edu/data/glove.6B.zip) [see Lab 7]
* Print the first line of the embeddings (see below) 

In [66]:
#your code goes here
import numpy as np
dic_targetVol={}
dic_sourceVol={}
#create the dictionary of source vocabulary and target vocabulary, the key are sentece and the value are index 
dic_sourceVol = dict([(char,i) for i, char in enumerate(Source_vocabulary)])
dic_targetVol= dict([(char,i) for i, char in enumerate(target_vocabulary)])
print(len(dic_sourceVol))

1833


In [99]:
#create the embedings for each sentence, here I use glove.6B.50d, for each line get the 
#word and embeding features for this word
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() 
        word2vector = {}
        for line in f:
            line_ = line.strip() 
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector
#the vocab is all words in this file and the w2v is a diactionary, the key is each word and
#the vlaue is feature embeding for each word.then firstly check whether this word in the vocab,
#then get the embeddings of this word 
def get_embedings(dic_sourceVol):
  vocab, w2v =read_data("/content/drive/My Drive/Colab Notebooks/glove.6B.50d.txt")
  print(w2v['good'])
  embeddings_source={}
  for val,key in dic_sourceVol.items():
      if val in vocab:
        embeddings_source[key]=w2v[val]
      else:
        embeddings_source[key]=np.zeros(len(embeddings_source[0]),dtype='float32')
 
  return embeddings_source

embeddings_source=get_embedings(dic_sourceVol) 
#this is the first embeding
# print(embeddings_source[0])

Total Words in DataSet: 400000
[-3.5586e-01  5.2130e-01 -6.1070e-01 -3.0131e-01  9.4862e-01 -3.1539e-01
 -5.9831e-01  1.2188e-01 -3.1943e-02  5.5695e-01 -1.0621e-01  6.3399e-01
 -4.7340e-01 -7.5895e-02  3.8247e-01  8.1569e-02  8.2214e-01  2.2220e-01
 -8.3764e-03 -7.6620e-01 -5.6253e-01  6.1759e-01  2.0292e-01 -4.8598e-02
  8.7815e-01 -1.6549e+00 -7.7418e-01  1.5435e-01  9.4823e-01 -3.9520e-01
  3.7302e+00  8.2855e-01 -1.4104e-01  1.6395e-02  2.1115e-01 -3.6085e-02
 -1.5587e-01  8.6583e-01  2.6309e-01 -7.1015e-01 -3.6770e-02  1.8282e-03
 -1.7704e-01  2.7032e-01  1.1026e-01  1.4133e-01 -5.7322e-02  2.7207e-01
  3.1305e-01  9.2771e-01]


## Section 2 Translation Model training



---



**Task 3 (20 marks)**
* Provide code for the encoder using Keras LSTM (5 marks)
* Provide code for the decoder using Keras LSTM (5 marks)
* Train the sequence2sequence (encoder-decoder) model (10 marks)


In [0]:
# encoder code goes here
# import numpy as np
#for the encoder and decoder, here firsly I use the embeding features as encoder, the generation way is 
# define the encoder input data shape. the length is smaple numers, for each sample the length is the max encoder
#sequence length and for each word the length is embeding feature length. The according to each words in the input text to
#get their embeding features, rest of length filled by zero.
def get_encoder_input_data(input_texts,max_encoder_seq_length,embeddings_source):
    encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,len(embeddings_source[0])),dtype='float32')
    for sen in range(len(input_texts)):
        source_tokenize1=input_texts[sen]
        for words in range(len(source_tokenize1)):
                  for i in range(len(embeddings_source[0])):
                      encoder_input_data[sen,words,i]=embeddings_source[dic_sourceVol[source_tokenize1[words].lower()]][i]
                      # embeddings_source[dic_sourceVol[source_tokenize1[word].lower()]][i]
    return encoder_input_data    
# for i,input_text in enumerate(input_texts):
#   for t, char in enumerate(input_text):
#       encoder_input_data[i,t,dic_sourceVol[char.lower()]]=1    
# print(encoder_input_data[0])
encoder_input_data=get_encoder_input_data(input_texts,max_encoder_seq_length,embeddings_source)


In [0]:
# decoder code goes here
# for the decoder data, here I use the one-hot method to get the decoder data, the decoder data has two part, first is input data, second is target data
# the target data is the next word of input data, for each word using one-hot to get the feature vector as input
def get_decoder_data(target_texts,max_decoder_seq_length,num_decoder_tokens):
    decoder_input_data=np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
    decoder_target_data=np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
    for i,target_text in enumerate(target_texts):
      for t, char in enumerate(target_text):
          decoder_input_data[i,t,dic_targetVol[char.lower()]]=1
          if t>0:
            decoder_target_data[i,t-1,dic_targetVol[char.lower()]]=1
    return decoder_input_data,decoder_target_data
  
decoder_input_data,decoder_target_data=get_decoder_data(target_texts,max_decoder_seq_length,num_decoder_tokens)
#shape of the decoder target data
print(decoder_target_data.shape)

In [70]:
#this is seq2seq network,the input is embedings of encoder data,add a LSTM layer to network
#then get the state, for hidden state and cell state, then using the hidden state and cell state as 
#the context and new input of decoder
batch_size=64
epochs= 30
latent_dim =256
encoder_inputs=Input(shape=(None,len(embeddings_source[0])))
encoder_lstm= LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c= encoder_lstm(encoder_inputs)
encoder_states= [state_h,state_c]

#the input is one-hot, the network structure is same with encoder, add input layer, one LSTM layer and 
#one dense layer, feed the input decoder data and encoder states.
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_output,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs=decoder_dense(decoder_output)

model= Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, None, 50)     0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, None, 3948)   0                                            
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, 256), (None, 314368      input_19[0][0]                   
__________________________________________________________________________________________________
lstm_12 (LSTM)                  [(None, None, 256),  4305920     input_20[0][0]                   
                                                                 lstm_11[0][1]                    
          

In [71]:
import sklearn
from sklearn.model_selection import train_test_split
#generate the train and test dataset, here I using train_test_split to split it, the split size is 0.2, the output below is shape of training and testing data
X_train_encoder_input_data,X_test_encoder_input_data,y_train_decoder_target_data,y_test_decoder_target_data=train_test_split(encoder_input_data,decoder_target_data,test_size=0.2,random_state=0)
X_train_decoder_input_data,X_test_decoder_input_data=train_test_split(decoder_input_data,test_size=0.2,random_state=0)
train_input_text,test_input_text,train_target_text,test_target_text=train_test_split(input_texts,target_texts,test_size=0.2,random_state=0)
print(X_train_encoder_input_data.shape)
print(X_test_encoder_input_data.shape)
print(X_train_decoder_input_data.shape)
print(X_test_decoder_input_data.shape)
print(y_train_decoder_target_data.shape)
print(y_test_decoder_target_data.shape)

(6400, 6, 50)
(1600, 6, 50)
(6400, 13, 3948)
(1600, 13, 3948)
(6400, 13, 3948)
(1600, 13, 3948)


In [72]:
#fitting data and saving data.
model.fit([X_train_encoder_input_data, X_train_decoder_input_data], y_train_decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('seq2seq_source_target.h5')

Train on 5120 samples, validate on 1280 samples
Epoch 1/30
5120/5120 [==============================] - 7s 1ms/step - loss: 1.9069 - val_loss: 1.7652
Epoch 2/30
5120/5120 [==============================] - 4s 828us/step - loss: 1.6183 - val_loss: 1.6456
Epoch 3/30
5120/5120 [==============================] - 4s 824us/step - loss: 1.4517 - val_loss: 1.5099
Epoch 4/30
5120/5120 [==============================] - 4s 832us/step - loss: 1.3070 - val_loss: 1.4147
Epoch 5/30
5120/5120 [==============================] - 4s 830us/step - loss: 1.2036 - val_loss: 1.3643
Epoch 6/30
5120/5120 [==============================] - 4s 830us/step - loss: 1.1263 - val_loss: 1.3164
Epoch 7/30
5120/5120 [==============================] - 4s 854us/step - loss: 1.0619 - val_loss: 1.2703
Epoch 8/30
5120/5120 [==============================] - 4s 874us/step - loss: 1.0060 - val_loss: 1.2405
Epoch 9/30
5120/5120 [==============================] - 4s 851us/step - loss: 0.9536 - val_loss: 1.2189
Epoch 10/30
5120/5

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_12 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_11/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_11/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


# Section 3 Testing


---


** Task 4 (20 marks) **


---



*   Use the trained model to translate the text from source into target language (10 marks). Use the test/evaluation set (see Section 1) and perform an automatic evaluation with the BLEU metric (10 marks). use the NLTK library to calculate BLEU.




In [0]:
#get the model and compile and load it
model= Model(inputs=[encoder_inputs,decoder_inputs],outputs=decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.load_weights('seq2seq_source_target.h5')

In [91]:
# #Your code goes here
#for the testing, here I use the decode_sequence function, using the model to get the
#encoder and decoder, according to the decoder prediction, get the index of each word and then
#using reverse_target_char_index to get the corresponsing word, if the words number greater than
#max sequence number ot its \n then break the loop, then get the decoder sentence.
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in dic_sourceVol.items())
reverse_target_char_index = dict(
  (i, char) for char, i in dic_targetVol.items())

def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, dic_targetVol['\t']] = 1.
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += ' '+sampled_char
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
      
    # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    # update states
    states_value = [h, c]   
  return decoded_sentence

for seq_index in range(10):
  input_seq = X_test_encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', test_input_text[seq_index])
  decoded_sentence=nltk.word_tokenize(decoded_sentence)
  print('Decoded sentence:', decoded_sentence)
  if ('\t' in test_target_text[seq_index] and '\n' in test_target_text[seq_index]):
    test_target_text[seq_index].remove('\t')
    test_target_text[seq_index].remove('\n')
  reference=[i.lower() for i in test_target_text[seq_index]]
  candidate=decoded_sentence
  print('Target sentence:', reference)
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], candidate)
  BLEUscore2 = nltk.translate.bleu_score.sentence_bleu([reference], candidate,weights=(0.25,0.25,0.25,0.25))
  print('BLEU score: ', BLEUscore)
  print('Ngram BLEU score: ', BLEUscore2)

  
#testing 
#this is for all testing data, the BLEU score use 4-gram, and caculate the sum of each gram, the average 
#BLEU score is 0.47963007282241854
BLEUscore_testing=0
BLEUscore2_testing=0

for seq_index in range(len(X_test_encoder_input_data)):
  pos_same_sentence=[]
  input_seq = X_test_encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  decoded_sentence=nltk.word_tokenize(decoded_sentence)
  if ('\t' in test_target_text[seq_index] and '\n' in test_target_text[seq_index]):
    test_target_text[seq_index].remove('\t')
    test_target_text[seq_index].remove('\n')
  reference=[i.lower() for i in test_target_text[seq_index]]
  candidate=decoded_sentence
  BLEUscore2_testing =BLEUscore2_testing+nltk.translate.bleu_score.sentence_bleu([reference], candidate,weights=(0.25,0.25,0.25,0.25))
aver_BLEUscore2=BLEUscore2_testing/(len(X_test_encoder_input_data))
print('Ngram BLEU score: ', aver_BLEUscore2)
  

-
Input sentence: ['You', "'re", 'nuts', '!']
Decoded sentence: ['vous', 'êtes', 'dingue']
Target sentence: ["t'es", 'givrée', '!']
BLEU score:  0
Ngram BLEU score:  0
-
Input sentence: ['Tom', 'snores', '.']
Decoded sentence: ['tom', 'a', 'parlé', '.']
Target sentence: ['tom', 'ronfle', '.']
BLEU score:  0.8408964152537145
Ngram BLEU score:  0.8408964152537145
-
Input sentence: ['They', "'re", 'lying', '.']
Decoded sentence: ['elles', 'sont', '.']
Target sentence: ['elles', 'mentent', '.']
BLEU score:  0.9036020036098448
Ngram BLEU score:  0.9036020036098448
-
Input sentence: ['Go', 'get', 'it', '.']
Decoded sentence: ['va', 'tout', 'changer']
Target sentence: ['va', 'le', 'chercher', '!']
BLEU score:  0.5444460596606694
Ngram BLEU score:  0.5444460596606694
-
Input sentence: ['Come', 'at', 'once', '.']
Decoded sentence: ['venez', 'ici', '.']
Target sentence: ['viens', 'ici', 'immédiatement', '.']
BLEU score:  0.6474591278836639
Ngram BLEU score:  0.6474591278836639
-
Input sentence: 

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Ngram BLEU score:  0.47963007282241854


# Section 4 Attention

---



** Task 5 (40 Marks) **sequence2sequence

* Extend the existing Seq2Seq model with an attention mechanism [Discussed in Class]
* Create sequence2sequence model with attention (15 marks)
* Train the model with the same data from Section 1 (10 marks)
* Translate the evaluation using the sequence2sequence attention model (10 marks)
* Evaluate the translations made with the sequence2sequence attention model and compare it with the model without attention using BLEU (5 marks)

In [0]:
#for the attention model, the first part is almost same with previous model
#the input shape of encoder and decoder are same to previous one, here using the
#feed the encoder and decoder to LSTM and then return sequences, get the initial_state with 
#decoder input data, then feed them into the LSTM as decoder
from keras.layers import SimpleRNN
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np

from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense
from keras.models import Model, load_model

encoder_input=Input(shape=(None,len(embeddings_source[0])))
decoder_input=Input(shape=(None,num_decoder_tokens))

encoder=LSTM(latent_dim, return_sequences=True)(encoder_input)
encoder_last = encoder[:,-1,:]

decoder=LSTM(latent_dim, return_sequences=True)(decoder_input, initial_state=[encoder_last, encoder_last])


In [86]:
#reference: https://github.com/wanasit/katakana/blob/master/notebooks/Attention-based%20Sequence-to-Sequence%20in%20Keras.ipynb
#Attention
# here I use the coding of reference above, the formula comes from 
# reference:https://arxiv.org/pdf/1508.04025.pdf
from keras.layers import Activation, dot, concatenate
# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)
# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(latent_dim, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))(output)
print('output', output)

attention Tensor("attention_5/truediv:0", shape=(?, ?, ?), dtype=float32)
context Tensor("dot_12/MatMul:0", shape=(?, ?, 256), dtype=float32)
decoder_combined_context Tensor("concatenate_6/concat:0", shape=(?, ?, 512), dtype=float32)
output Tensor("time_distributed_12/Reshape_1:0", shape=(?, ?, 3948), dtype=float32)


In [87]:
#compile the model
model_attention = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model_attention.compile(optimizer='adam', loss='categorical_crossentropy')
model_attention.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, None, 3948)   0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           (None, None, 50)     0                                            
__________________________________________________________________________________________________
lstm_16 (LSTM)                  (None, None, 256)    4305920     input_26[0][0]                   
__________________________________________________________________________________________________
lstm_15 (LSTM)                  (None, None, 256)    314368      input_25[0][0]                   
__________________________________________________________________________________________________
dot_11 (Do

In [88]:
#training the model  
model_attention.fit([X_train_encoder_input_data, X_train_decoder_input_data], y_train_decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model_attention.save('seq2seq_source_target_attention.h5')

Train on 5120 samples, validate on 1280 samples
Epoch 1/30
5120/5120 [==============================] - 9s 2ms/step - loss: 2.1478 - val_loss: 1.9651
Epoch 2/30
5120/5120 [==============================] - 4s 838us/step - loss: 1.8440 - val_loss: 1.9113
Epoch 3/30
5120/5120 [==============================] - 4s 836us/step - loss: 1.7656 - val_loss: 1.8416
Epoch 4/30
5120/5120 [==============================] - 4s 842us/step - loss: 1.6673 - val_loss: 1.7750
Epoch 5/30
5120/5120 [==============================] - 4s 845us/step - loss: 1.6014 - val_loss: 1.7291
Epoch 6/30
5120/5120 [==============================] - 4s 846us/step - loss: 1.5490 - val_loss: 1.6874
Epoch 7/30
5120/5120 [==============================] - 5s 887us/step - loss: 1.5001 - val_loss: 1.6494
Epoch 8/30
5120/5120 [==============================] - 5s 882us/step - loss: 1.4556 - val_loss: 1.6110
Epoch 9/30
5120/5120 [==============================] - 4s 841us/step - loss: 1.4073 - val_loss: 1.5798
Epoch 10/30
5120/5

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_16 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'strided_slice_4:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'strided_slice_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [0]:
#get the model and compile and load it
model_attention= Model(inputs=[encoder_input, decoder_input], outputs=[output])
model_attention.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model_attention.load_weights('seq2seq_source_target_attention.h5')

In [93]:
# #Your code goes here
#for the testing, here I use model.predict directly, the input data
#are testing encoder data and testing decoder data, same to previous one
#here I can get the index of each output, and then using recerse_target_char_index to reverse
#the index to word, then tokenize it and compare the sentence with original sentence, 
#then using nltk.bleu to get score, the final score is 0.5336994507415952, it improved
# nearly 6%.
def attention_testing():
  decoded_sentences=[]
  predictions=model_attention.predict([X_test_encoder_input_data, X_test_decoder_input_data])
  for i in range(predictions.shape[0]):
    decoded_sentence=''
    for j in range(predictions.shape[1]):
        output_tokens=predictions[i][j]
        sampled_token_index2 = np.argmax(output_tokens)
        sampled_char2 = reverse_target_char_index[sampled_token_index2]  
        if (sampled_char2 == '\n' or len(decoded_sentence) > max_decoder_seq_length):
          break
        decoded_sentence += ' '+sampled_char2
        
    decoded_sentences.append(decoded_sentence)
  return decoded_sentences
decoded_sentences=attention_testing()
BLEUscore2_testing2=0
for i in range(len(decoded_sentences)):
    reference=[j.lower() for j in test_target_text[i]]
    decoded_sentence2=nltk.word_tokenize(decoded_sentences[i])
    candidate=decoded_sentence2
    BLEUscore2_testing2 =BLEUscore2_testing2+nltk.translate.bleu_score.sentence_bleu([reference], candidate,weights=(0.25,0.25,0.25,0.25))
aver_BLEUscore2=BLEUscore2_testing2/(len(X_test_encoder_input_data))
print('Ngram BLEU score: ', aver_BLEUscore2)  

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Ngram BLEU score:  0.5336994507415952
